In [1]:
#Script to compile index list and generate daily IRS report
import datetime
import requests
import os.path
import os
from zipfile import ZipFile
import csv
import psycopg2
import pandas as pd
import calendar
import numpy as np
import pandas.io.sql as sqlio
import time
import math
from datetime import timedelta
from utils.db_helper import DB_Helper
import utils.date_set as date_set

C:\Users\dsram\AppData\Local\Temp\ipykernel_15708\3176535713.py:9: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
conn = DB_Helper().db_connect()
cur = conn.cursor()
dbURL = DB_Helper().engine()

Connection to database BravisaDB successful.


In [16]:
def gen_sub_industry_index(conn, date):
    """ Generate the Industry index

    Operation:
        Fetch the data from IndustryDivisor and IndustryList for current date,
        and calculate the Open, close, high, low, PE , EPS and Earning Growth,

        'open' = ff_open/divisor, 'high' = ff_high/divisor, 'low' = ff_low/divisor,

        'close' = ff_close/divisor, 'earnings growth' = (eps – eps back / eps back) * 100 ,

        'pe' = sum os close / ff earning, eps = ff earning / ff equity.

    Return:
        Data of Industry index.
    """
      # Generating Industry index list

    industry_divisor_sql = 'SELECT * FROM public."SubIndustryDivisor" WHERE "Date" = \''+date+'\' ;'
    industry_divisor = sqlio.read_sql_query(industry_divisor_sql, con = conn)

    industry_ff_ohlc_sql = 'SELECT "SubIndustryIndexName", "SubIndustry", \
                        SUM("OS_Close") AS sum_os_close,\
                        SUM("FF_Open") AS ff_open_sum, \
                        SUM("FF_High") AS ff_high_sum, \
                        SUM("FF_Low") AS ff_low_sum, \
                        SUM("FF_Close") AS ff_close_sum, \
                        SUM("Volume") AS sum_vol, \
                        SUM("OS") AS os_sum, \
                        SUM("PAT") AS earnings, \
                        SUM("Equity") AS sum_equity,   \
                        SUM("prev_pat") AS prev_earnings, \
                        SUM("prev_equity") AS prev_equity, \
                        count(*) AS company_count\
                        FROM public."IndustryList" \
                        WHERE "GenDate" = \''+date+'\' AND "SubIndustryIndexName" is not null \
                        GROUP BY "SubIndustryIndexName", "SubIndustry" ;'
    industry_ff_ohlc = sqlio.read_sql_query(industry_ff_ohlc_sql, con = conn)

    for index, row in industry_ff_ohlc.iterrows():

      ff_open_list = industry_ff_ohlc.loc[(industry_ff_ohlc['SubIndustryIndexName']==row['SubIndustryIndexName'])]['ff_open_sum']
      ff_open = ff_open_list.item() if len(ff_open_list.index) == 1 else np.nan

      ff_high_list = industry_ff_ohlc.loc[(industry_ff_ohlc['SubIndustryIndexName']==row['SubIndustryIndexName'])]['ff_high_sum']
      ff_high = ff_high_list.item() if len(ff_high_list.index) == 1 else np.nan

      ff_low_list = industry_ff_ohlc.loc[(industry_ff_ohlc['SubIndustryIndexName']==row['SubIndustryIndexName'])]['ff_low_sum']
      ff_low = ff_low_list.item() if len(ff_low_list.index) == 1 else np.nan

      ff_close_list = industry_ff_ohlc.loc[(industry_ff_ohlc['SubIndustryIndexName']==row['SubIndustryIndexName'])]['ff_close_sum']
      ff_close = ff_close_list.item() if len(ff_close_list.index) == 1 else np.nan

      ff_os_list = industry_ff_ohlc.loc[(industry_ff_ohlc['SubIndustryIndexName']==row['SubIndustryIndexName'])]['os_sum']
      ff_os = ff_os_list.item() if len(ff_os_list.index) == 1 else np.nan

      ff_earnings_list = industry_ff_ohlc.loc[(industry_ff_ohlc['SubIndustryIndexName']==row['SubIndustryIndexName'])]['earnings']
      ff_earnings = ff_earnings_list.item() if len(ff_earnings_list.index) == 1 else np.nan

      ff_equity_list = industry_ff_ohlc.loc[(industry_ff_ohlc['SubIndustryIndexName']==row['SubIndustryIndexName'])]['sum_equity']
      ff_equity = ff_equity_list.item() if len(ff_equity_list.index) == 1 else np.nan

      prev_earnings_list = industry_ff_ohlc.loc[(industry_ff_ohlc['SubIndustryIndexName']==row['SubIndustryIndexName'])]['prev_earnings']
      prev_earnings = prev_earnings_list.item() if len(prev_earnings_list.index) == 1 else np.nan

      prev_equity_list = industry_ff_ohlc.loc[(industry_ff_ohlc['SubIndustryIndexName']==row['SubIndustryIndexName'])]['prev_equity']
      prev_equity = prev_equity_list.item() if len(prev_equity_list.index) == 1 else np.nan

      divisor_list = industry_divisor.loc[(industry_divisor['IndexName']==row['SubIndustryIndexName'])]['Divisor']
      divisor = divisor_list.item() if len(divisor_list.index) == 1 else np.nan

      sum_os_close_list = industry_ff_ohlc.loc[(industry_ff_ohlc['SubIndustryIndexName']==row['SubIndustryIndexName'])]['sum_os_close']
      sum_os_close = sum_os_close_list.item() if len(sum_os_close_list.index) == 1 else np.nan

      if np.isnan(divisor) or divisor == 0:
        divisor  =  10000
        print("Devisor gen_Subindustry_index: ", divisor)

      div_open = ff_open/divisor if divisor != 0 and not np.isnan(ff_open) else np.nan
      div_high = ff_high/divisor if divisor != 0 and not np.isnan(ff_high) else np.nan
      div_low = ff_low/divisor if divisor != 0 and not np.isnan(ff_low) else np.nan
      div_close = ff_close/divisor if divisor != 0 and not np.isnan(ff_close) else np.nan


      # div_open = ff_open/divisor if divisor != 0 and not np.isnan(ff_open) else np.nan
      # div_open = ff_open/divisor if divisor and ff_open is not None and divisor != 0 else np.nan
      # div_high = ff_high/divisor if divisor and ff_high is not None and divisor != 0 else np.nan
      # div_low = ff_low/divisor if divisor and ff_low is not None and divisor != 0 else np.nan
      # div_close = ff_close/divisor if divisor and ff_close is not None and divisor != 0 else np.nan


      pe = (sum_os_close)/ff_earnings if ff_earnings !=0 else np.nan
      eps = ff_earnings/ff_equity if ff_equity !=0 else np.nan
      eps_back = prev_earnings/prev_equity if prev_equity !=0 else np.nan
      earnings_growth = ((eps-eps_back)/abs(eps_back))*100 if eps_back != 0 else np.nan


      industry_ff_ohlc.loc[index, 'Open'] = div_open
      industry_ff_ohlc.loc[index, 'High'] = div_high
      industry_ff_ohlc.loc[index, 'Low'] = div_low
      industry_ff_ohlc.loc[index, 'Close'] = div_close
      industry_ff_ohlc.loc[index, 'PE'] = pe
      industry_ff_ohlc.loc[index, 'EPS'] = eps
      industry_ff_ohlc.loc[index, 'Earnings Growth'] = earnings_growth


    return industry_ff_ohlc

In [3]:
def gen_industry_index(conn, date):
    """ Generate the Industry index

    Operation:
        Fetch the data from IndustryDivisor and IndustryList for current date,
        and calculate the Open, close, high, low, PE , EPS and Earning Growth,

        'open' = ff_open/divisor, 'high' = ff_high/divisor, 'low' = ff_low/divisor,

        'close' = ff_close/divisor, 'earnings growth' = (eps – eps back / eps back) * 100 ,

        'pe' = sum os close / ff earning, eps = ff earning / ff equity.

    Return:
        Data of Industry index.
    """
      # Generating Industry index list

    industry_divisor_sql = 'SELECT * FROM public."IndustryDivisor" WHERE "Date" = \''+date+'\' ;'
    industry_divisor = sqlio.read_sql_query(industry_divisor_sql, con = conn)

    industry_ff_ohlc_sql = 'SELECT "IndustryIndexName", "Industry", \
                        SUM("OS_Close") AS sum_os_close,\
                        SUM("FF_Open") AS ff_open_sum, \
                        SUM("FF_High") AS ff_high_sum, \
                        SUM("FF_Low") AS ff_low_sum, \
                        SUM("FF_Close") AS ff_close_sum, \
                        SUM("Volume") AS sum_vol, \
                        SUM("OS") AS os_sum, \
                        SUM("PAT") AS earnings, \
                        SUM("Equity") AS sum_equity,   \
                        SUM("prev_pat") AS prev_earnings, \
                        SUM("prev_equity") AS prev_equity, \
                        count(*) AS company_count\
                        FROM public."IndustryList" \
                        WHERE "GenDate" = \''+date+'\' AND "IndustryIndexName" is not null \
                        GROUP BY "IndustryIndexName", "Industry" ;'
    industry_ff_ohlc = sqlio.read_sql_query(industry_ff_ohlc_sql, con = conn)

    for index, row in industry_ff_ohlc.iterrows():

      ff_open_list = industry_ff_ohlc.loc[(industry_ff_ohlc['IndustryIndexName']==row['IndustryIndexName'])]['ff_open_sum']
      ff_open = ff_open_list.item() if len(ff_open_list.index) == 1 else np.nan

      ff_high_list = industry_ff_ohlc.loc[(industry_ff_ohlc['IndustryIndexName']==row['IndustryIndexName'])]['ff_high_sum']
      ff_high = ff_high_list.item() if len(ff_high_list.index) == 1 else np.nan

      ff_low_list = industry_ff_ohlc.loc[(industry_ff_ohlc['IndustryIndexName']==row['IndustryIndexName'])]['ff_low_sum']
      ff_low = ff_low_list.item() if len(ff_low_list.index) == 1 else np.nan

      ff_close_list = industry_ff_ohlc.loc[(industry_ff_ohlc['IndustryIndexName']==row['IndustryIndexName'])]['ff_close_sum']
      ff_close = ff_close_list.item() if len(ff_close_list.index) == 1 else np.nan

      ff_os_list = industry_ff_ohlc.loc[(industry_ff_ohlc['IndustryIndexName']==row['IndustryIndexName'])]['os_sum']
      ff_os = ff_os_list.item() if len(ff_os_list.index) == 1 else np.nan

      ff_earnings_list = industry_ff_ohlc.loc[(industry_ff_ohlc['IndustryIndexName']==row['IndustryIndexName'])]['earnings']
      ff_earnings = ff_earnings_list.item() if len(ff_earnings_list.index) == 1 else np.nan

      ff_equity_list = industry_ff_ohlc.loc[(industry_ff_ohlc['IndustryIndexName']==row['IndustryIndexName'])]['sum_equity']
      ff_equity = ff_equity_list.item() if len(ff_equity_list.index) == 1 else np.nan

      prev_earnings_list = industry_ff_ohlc.loc[(industry_ff_ohlc['IndustryIndexName']==row['IndustryIndexName'])]['prev_earnings']
      prev_earnings = prev_earnings_list.item() if len(prev_earnings_list.index) == 1 else np.nan

      prev_equity_list = industry_ff_ohlc.loc[(industry_ff_ohlc['IndustryIndexName']==row['IndustryIndexName'])]['prev_equity']
      prev_equity = prev_equity_list.item() if len(prev_equity_list.index) == 1 else np.nan

      divisor_list = industry_divisor.loc[(industry_divisor['IndexName']==row['IndustryIndexName'])]['Divisor']
      divisor = divisor_list.item() if len(divisor_list.index) == 1 else np.nan

      sum_os_close_list = industry_ff_ohlc.loc[(industry_ff_ohlc['IndustryIndexName']==row['IndustryIndexName'])]['sum_os_close']
      sum_os_close = sum_os_close_list.item() if len(sum_os_close_list.index) == 1 else np.nan

      if np.isnan(divisor) or divisor == 0:
        print("Devisor gen_industry_index: ", divisor)


      div_open = ff_open/divisor if divisor != 0 and not np.isnan(ff_open) else np.nan
      div_high = ff_high/divisor if divisor != 0 and not np.isnan(ff_high) else np.nan
      div_low = ff_low/divisor if divisor != 0 and not np.isnan(ff_low) else np.nan
      div_close = ff_close/divisor if divisor != 0 and not np.isnan(ff_close) else np.nan


      pe = (sum_os_close)/ff_earnings if ff_earnings !=0 else np.nan
      eps = ff_earnings/ff_equity if ff_equity !=0 else np.nan
      eps_back = prev_earnings/prev_equity if prev_equity !=0 else np.nan
      earnings_growth = ((eps-eps_back)/abs(eps_back))*100 if eps_back != 0 else np.nan


      industry_ff_ohlc.loc[index, 'Open'] = div_open
      industry_ff_ohlc.loc[index, 'High'] = div_high
      industry_ff_ohlc.loc[index, 'Low'] = div_low
      industry_ff_ohlc.loc[index, 'Close'] = div_close
      industry_ff_ohlc.loc[index, 'PE'] = pe
      industry_ff_ohlc.loc[index, 'EPS'] = eps
      industry_ff_ohlc.loc[index, 'Earnings Growth'] = earnings_growth


    return industry_ff_ohlc

In [7]:
date = datetime.datetime.strptime('2020-01-01', "%Y-%m-%d").date()

In [11]:
ind_ind = gen_industry_index(conn,'2020-01-01')

Devisor gen_industry_index:  nan
Devisor gen_industry_index:  nan
Devisor gen_industry_index:  nan
Devisor gen_industry_index:  nan
Devisor gen_industry_index:  nan
Devisor gen_industry_index:  nan
Devisor gen_industry_index:  nan
Devisor gen_industry_index:  nan
Devisor gen_industry_index:  nan
Devisor gen_industry_index:  nan
Devisor gen_industry_index:  nan
Devisor gen_industry_index:  nan
Devisor gen_industry_index:  nan
Devisor gen_industry_index:  nan
Devisor gen_industry_index:  nan
Devisor gen_industry_index:  nan
Devisor gen_industry_index:  nan
Devisor gen_industry_index:  nan
Devisor gen_industry_index:  nan
Devisor gen_industry_index:  nan
Devisor gen_industry_index:  nan
Devisor gen_industry_index:  nan
Devisor gen_industry_index:  nan
Devisor gen_industry_index:  nan
Devisor gen_industry_index:  nan
Devisor gen_industry_index:  nan
Devisor gen_industry_index:  nan
Devisor gen_industry_index:  nan
Devisor gen_industry_index:  nan
Devisor gen_industry_index:  nan
Devisor ge

C:\Users\dsram\AppData\Local\Temp\ipykernel_15708\3753011609.py:20: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  industry_divisor = sqlio.read_sql_query(industry_divisor_sql, con = conn)
C:\Users\dsram\AppData\Local\Temp\ipykernel_15708\3753011609.py:38: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  industry_ff_ohlc = sqlio.read_sql_query(industry_ff_ohlc_sql, con = conn)


In [17]:
sub_ind = gen_sub_industry_index(conn, '2020-01-01')

C:\Users\dsram\AppData\Local\Temp\ipykernel_15708\1608621548.py:20: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  industry_divisor = sqlio.read_sql_query(industry_divisor_sql, con = conn)
C:\Users\dsram\AppData\Local\Temp\ipykernel_15708\1608621548.py:38: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  industry_ff_ohlc = sqlio.read_sql_query(industry_ff_ohlc_sql, con = conn)


Devisor gen_Subindustry_index:  10000
Devisor gen_Subindustry_index:  10000
Devisor gen_Subindustry_index:  10000
Devisor gen_Subindustry_index:  10000
Devisor gen_Subindustry_index:  10000
Devisor gen_Subindustry_index:  10000
Devisor gen_Subindustry_index:  10000
Devisor gen_Subindustry_index:  10000
Devisor gen_Subindustry_index:  10000
Devisor gen_Subindustry_index:  10000
Devisor gen_Subindustry_index:  10000
Devisor gen_Subindustry_index:  10000
Devisor gen_Subindustry_index:  10000
Devisor gen_Subindustry_index:  10000
Devisor gen_Subindustry_index:  10000
Devisor gen_Subindustry_index:  10000
Devisor gen_Subindustry_index:  10000
Devisor gen_Subindustry_index:  10000
Devisor gen_Subindustry_index:  10000
Devisor gen_Subindustry_index:  10000
Devisor gen_Subindustry_index:  10000
Devisor gen_Subindustry_index:  10000
Devisor gen_Subindustry_index:  10000
Devisor gen_Subindustry_index:  10000
Devisor gen_Subindustry_index:  10000
Devisor gen_Subindustry_index:  10000
Devisor gen_

In [18]:
sub_ind

,SubIndustryIndexName,SubIndustry,sum_os_close,ff_open_sum,ff_high_sum,ff_low_sum,ff_close_sum,sum_vol,os_sum,earnings,...,prev_earnings,prev_equity,company_count,Open,High,Low,Close,PE,EPS,Earnings Growth
0,AALG,"Apparel, Accessories & Luxury Goods",1.877216e+10,7.730535e+09,7.799818e+09,7.622964e+09,7.787055e+09,51510.0,8.790520e+07,1.476954e+09,...,1.830642e+09,4.532760e+08,2,7.730535e+05,7.799818e+05,7.622964e+05,7.787055e+05,12.710048,3.258399,-19.320432
1,ACJ,Auto - Cars & Jeeps,6.084997e+12,2.985920e+12,3.004446e+12,2.952787e+12,2.974029e+12,14810400.0,3.354138e+09,2.267312e+11,...,2.627133e+11,1.755234e+10,8,2.985920e+08,3.004446e+08,2.952787e+08,2.974029e+08,26.837936,12.904010,-13.786048
2,AFM,Agricultural & Farm Machinery,1.107996e+10,8.000028e+09,8.147268e+09,7.803708e+09,8.147268e+09,4138830.0,1.334935e+09,-3.954600e+09,...,6.232600e+09,2.062600e+09,2,8.000028e+05,8.147268e+05,7.803708e+05,8.147268e+05,-2.801791,-1.917289,-163.450244
3,ALCVHCV,Auto - LCVs/HCVs,1.760833e+12,9.182340e+11,9.243273e+11,9.091719e+11,9.133062e+11,67723076.0,1.262875e+10,-2.420180e+10,...,7.189132e+10,2.022800e+10,10,9.182340e+07,9.243273e+07,9.091719e+07,9.133062e+07,-72.756287,-1.150582,-132.373828
4,ALU,Aluminum,1.121842e+12,7.063611e+11,7.107654e+11,6.962669e+11,7.000900e+11,26435716.0,8.167697e+09,1.596920e+10,...,6.204900e+10,2.311500e+10,4,7.063611e+07,7.107654e+07,6.962669e+07,7.000900e+07,70.250356,0.691421,-74.242617
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,THSP,"Technology Hardware, Storage & Peripherals",7.485291e+09,3.578976e+09,3.578976e+09,3.477670e+09,3.538152e+09,7156.0,3.198842e+07,6.710240e+08,...,5.918120e+08,3.266340e+08,2,3.578976e+05,3.578976e+05,3.477670e+05,3.538152e+05,11.155028,2.054361,13.384662
95,TOB,Tobacco,6.207424e+12,6.071662e+12,6.074489e+12,6.034270e+12,6.061253e+12,8571200.0,2.508609e+10,3.096478e+11,...,2.477192e+11,2.501060e+10,6,6.071662e+08,6.074489e+08,6.034270e+08,6.061253e+08,20.046726,12.337057,24.559254
96,TR,Tires & Rubber,1.345640e+12,7.793366e+11,7.862655e+11,7.762356e+11,7.813069e+11,2912744.0,2.329113e+09,6.222766e+10,...,7.120386e+10,4.332900e+09,14,7.793366e+07,7.862655e+07,7.762356e+07,7.813069e+07,21.624470,14.108981,-14.143976
97,TRA,Trading,5.452606e+11,1.439928e+11,1.450560e+11,1.423244e+11,1.431422e+11,3926690.0,5.328501e+09,2.376956e+10,...,7.096578e+09,5.656586e+09,6,1.439928e+07,1.450560e+07,1.423244e+07,1.431422e+07,22.939450,4.202104,234.943982


In [22]:
def calc_rank_index(industry_index, sub_industry_index,conn, date):
    """ calculating the rank for sector, subsector and industry.

    Args:
        sector_index = data of Open, close, high, low, PE , EPS
                      and Earning Growth of sector index.

        subsector_index = data of Open, close, high, low, PE , EPS
                        and Earning Growth of subsector index.

        industry_index = data of Open, close, high, low, PE , EPS
                        and Earning Growth of industry index.

    Operation:
        Fetch the data from IndexHistory and calculate the value for six month change
        for sector index, subsector index and industry index,
        change six month = (curr close - prev close / prev close)* 100
        index = change * 100.

    Return:
        Rank value of Sector Index, SubSector Index and Industry index.
    """

    six_month_back = '01-06-2019'

    indexlist_history_sql = 'SELECT DISTINCT ON("TICKER") * FROM public."IndexHistory" \
                            where "DATE" <= \''+six_month_back+'\' \
                            ORDER BY "TICKER", "DATE" DESC; '
    index_history = sqlio.read_sql_query(indexlist_history_sql, con = conn)


    #Sector rank
    # if not(sector_index.empty):

    #   for index, row in sector_index.iterrows():

    #     prev_close_list = index_history.loc[(index_history['TICKER']==row['SectorIndexName'])]['CLOSE']
    #     prev_close = prev_close_list.item() if len(prev_close_list.index) == 1 else np.nan

    #     current_close_list = sector_index.loc[(sector_index['SectorIndexName']==row['SectorIndexName'])]['Close']
    #     current_close = current_close_list.item() if len(current_close_list.index) == 1 else np.nan

    #     change_six_months = (current_close-prev_close)/prev_close * 100 if prev_close != np.nan else 0

    #     sector_index.loc[index, 'Change'] = change_six_months

    #   sector_index['Rank'] = sector_index['Change'].rank(ascending=True, pct=True) * 100

    #   sector_index = sector_index.rename(columns = {"SectorIndexName":"IndexName", "Sector": "Index"})

    # else:

    #   print("No Sector data")


    # #SubSector Rank
    # if not(subsector_index.empty):

    #   for index, row in subsector_index.iterrows():

    #     prev_close_list = index_history.loc[(index_history['TICKER']==row['SubSectorIndexName'])]['CLOSE']
    #     prev_close = prev_close_list.item() if len(prev_close_list.index) == 1 else np.nan

    #     current_close_list = subsector_index.loc[(subsector_index['SubSectorIndexName']==row['SubSectorIndexName'])]['Close']
    #     current_close = current_close_list.item() if len(current_close_list.index) == 1 else np.nan

    #     change_six_months = (current_close-prev_close)/prev_close * 100 if prev_close != np.nan else 0

    #     subsector_index.loc[index, 'Change'] = change_six_months

    #   subsector_index['Rank'] = subsector_index['Change'].rank(ascending=True, pct=True) * 100

    #   subsector_index = subsector_index.rename(columns = {"SubSectorIndexName":"IndexName", "SubSector": "Index"})

    # else:

    #   print("No Subsector data")


    #Industry Rank
    if not(industry_index.empty):

      for index, row in industry_index.iterrows():

        prev_close_list = index_history.loc[(index_history['TICKER']==row['IndustryIndexName'])]['CLOSE']
        prev_close = prev_close_list.item() if len(prev_close_list.index) == 1 else np.nan

        current_close_list = industry_index.loc[(industry_index['IndustryIndexName']==row['IndustryIndexName'])]['Close']
        current_close = current_close_list.item() if len(current_close_list.index) == 1 else np.nan

        change_six_months = (current_close-prev_close)/prev_close * 100 if prev_close != np.nan else 0

        industry_index.loc[index, 'Change'] = change_six_months

      industry_index['Rank'] = industry_index['Change'].rank(ascending=True, pct=True) * 100

      industry_index = industry_index.rename(columns = {"IndustryIndexName":"IndexName", "Industry": "Index"})

    else:

      print("No Industry data")
      
    #SubIndustry Rank
    if not(sub_industry_index.empty):
    
      for index, row in sub_industry_index.iterrows():

        prev_close_list = index_history.loc[(index_history['TICKER']==row['SubIndustryIndexName'])]['CLOSE']
        prev_close = prev_close_list.item() if len(prev_close_list.index) == 1 else np.nan

        current_close_list = sub_industry_index.loc[(sub_industry_index['SubIndustryIndexName']==row['SubIndustryIndexName'])]['Close']
        current_close = current_close_list.item() if len(current_close_list.index) == 1 else np.nan

        change_six_months = (current_close-prev_close)/prev_close * 100 if prev_close != np.nan else 0

        sub_industry_index.loc[index, 'Change'] = change_six_months

      sub_industry_index['Rank'] = sub_industry_index['Change'].rank(ascending=True, pct=True) * 100

      sub_industry_index = sub_industry_index.rename(columns = {"SubIndustryIndexName":"IndexName", "SubIndustry": "Index"})

    else:

      print("No SubIndustry data")


    irs_list = pd.concat([industry_index, sub_industry_index], axis=0, sort=True)
    print('irslist: ',irs_list)


    return irs_list

In [24]:
rank = calc_rank_index(ind_ind,sub_ind, conn, '01-01-2020')

irslist:            Change         Close        EPS  Earnings Growth          High  \
0            NaN           NaN   5.966344       -17.634167           NaN   
1            NaN           NaN   1.161483       277.990251           NaN   
2            NaN           NaN   9.276514       -22.187876           NaN   
3            NaN           NaN  -0.140947        89.095276           NaN   
4            NaN           NaN   1.994288       -49.114438           NaN   
..           ...           ...        ...              ...           ...   
94  5.173270e+04  3.538152e+05   2.054361        13.384662  3.578976e+05   
95  5.799733e+07  6.061253e+08  12.337057        24.559254  6.074489e+08   
96           NaN  7.813069e+07  14.108981       -14.143976  7.862655e+07   
97           NaN  1.431422e+07   4.202104       234.943982  1.450560e+07   
98           NaN  1.276951e+06   9.027977        18.614067  1.297469e+06   

                                         Index IndexName           Low  \
0  

C:\Users\dsram\AppData\Local\Temp\ipykernel_15708\2917508967.py:29: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  index_history = sqlio.read_sql_query(indexlist_history_sql, con = conn)
